In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


vinc

In [32]:
import pandas as pd
data = pd.read_csv('http://cocl.us/Geospatial_data')
data = pd.DataFrame(data)
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
all_tables=soup.findAll("table")
right_table = soup.find('table',{'class':'wikitable'})
A=[]
B=[]
C=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
import pandas as pd
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [34]:
df1=df.drop([0,1,7,10,15,16,19,24,25,28,29,33,34,35,37,38,42,43,44,51,52,53,60,61,62,69,70,71,78,79,87,88,96,97,101,105,106,110,115,118,119,123,124,125,127,128,131,132,133,134,136,137,140,141,145,146,149,150,154,155,158,159,161,162,163,164,166,167,170,171,172,173,174,175,176,177,179])
df1.reset_index(drop=True, inplace=True)
df2=df1.sort_values(by=['Postcode'])
df2.reset_index(drop=True, inplace=True)
df3 =df2.rename(columns ={'Postcode':'Postal Code'})

In [35]:
data.reset_index(drop=True, inplace=True)

df3.reset_index(drop=True, inplace=True)
Result = pd.concat([df3, data], axis=1)
Result.head()

,Postal Code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,M1B,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,M1C,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [27]:
for data in Result:
    borough =  Result['Borough'] 
    neighborhood_name = Result['Neighborhood']
        
    neighborhood_latlon = Result['Latitude'],Result['Longitude']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    Result = Result.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
Result

,Postal Code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,M1B,43.8067,-79.1944
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,M1C,43.7845,-79.1605
2,M1E,Scarborough,Guildwood / Morningside / West Hill,M1E,43.7636,-79.1887
3,M1G,Scarborough,Woburn,M1G,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,M1H,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,M1J,43.7447,-79.2395
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,M1K,43.7279,-79.262
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,M1L,43.7111,-79.2846
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village W...,M1M,43.7163,-79.2395
9,M1N,Scarborough,Birch Cliff / Cliffside West,M1N,43.6927,-79.2648


In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [21]:
CLIENT_ID = 'F3CI01K5004BYWF2UPRJP4ROQ5NX1DFIZ5WOVCXI2VOTDDSZ'
CLIENT_SECRET = 'KFVKLDRKEYVKODTEDS2FOTJYIN4VVYFYHNBSXGO12MVU0Q2Z' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F3CI01K5004BYWF2UPRJP4ROQ5NX1DFIZ5WOVCXI2VOTDDSZ
CLIENT_SECRET:KFVKLDRKEYVKODTEDS2FOTJYIN4VVYFYHNBSXGO12MVU0Q2Z


In [36]:
Toronto_data = Result[Result['Borough'] == 'Toronto'].reset_index(drop=True)
Toronto_data.head()

,Postal Code,Borough,Neighborhood,Postal Code,Latitude,Longitude


In [46]:
Result['Borough'] == 'Central Toronto'

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
30     False
31     False
32     False
33     False
34     False
35     False
36     False
37     False
38     False
39     False
40     False
41     False
42     False
43     False
44     False
45     False
46     False
47     False
48     False
49     False
50     False
51     False
52     False
53     False
54     False
55     False
56     False
57     False
58     False
59     False
60     False
61     False
62     False
63     False
64     False
65     False
66     False
67     False
68     False
69     False
70     False
71     False
72     False
73     False
74     False
75     False
76     False